In [77]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def nextTokenizedSentence(cursor, textFieldName):
    print("Calculating record count")
    results_count = cursor.count()
    print("Count: " + str(results_count))
    i = 0
    percent_old = None
    for text_doc in cursor:
        i=i+1
        #print(str(i))
        percent = '{:03.0f}'.format(i/results_count*100)
        if percent != percent_old:
            print(percent)
        percent_old = percent
        text = text_doc[textFieldName]
        for sentence in sent_detector.tokenize(text):
            tokens = word_tokenize(sentence.lower())
            yield [token for token in tokens if token not in stop and token not in exclude]



In [78]:
def nextTaggedSentence(cursor, textFieldName):
    for tokenizedSentence in nextTokenizedSentence(cursor, textFieldName):
        #print(tokenizedSentence)
        tagged = nltk.pos_tag(tokenizedSentence)
        yield tagged

In [79]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases



In [80]:
def nextNounPhrase(cursor, textFieldName):
    for taggedSentence in nextTaggedSentence(cursor, textFieldName):
        grammar = r"""
          NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
            """
        cp = nltk.RegexpParser(grammar)
        if taggedSentence:
            noun_phrase_tagged = cp.parse(taggedSentence)
            noun_phrases=[]
            nouns=traverse(noun_phrase_tagged,noun_phrases)
            #print(nouns)
            for noun in nouns:
                yield noun
 

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



In [81]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    cursor = my_collection.find() 
    textFieldName = "buyerTranslationFeedback"
else:
    my_collection = mydb['amazon_product_reviews']
    cursor = my_collection.find({ "categories" : "Cell Phones" })
    textFieldName = "reviewText"


In [82]:
from nltk import FreqDist


fdist = FreqDist(nextNounPhrase(cursor, textFieldName)) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)

Calculating record count
Count: 159702
000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
[('phone', 380958), ('battery', 40112), ('use', 39962), ('time', 33567), ('screen', 32044), ('phones', 31332), ('camera', 28181), ('card', 21266), ('service', 20690), ('price', 20230), ('work', 19194), ('quality', 18522), ('problem', 18242), ('life', 17437), ('iphone', 16087), ('product', 15630), ('day', 15213), ('thing', 14719), ('features', 14691), ('cell', 14281), ('way', 13705), ('minutes', 13698), ('everything', 13686), ('days', 13654), ('samsung', 13280), ('works', 12870), ('call', 12769), ('problems', 12733), ('apps', 12585), ('data', 12463), (

In [85]:
features=  [ {"feature" : key} for key in frequent_words]
print(features)

[{'feature': 'phone'}, {'feature': 'battery'}, {'feature': 'use'}, {'feature': 'time'}, {'feature': 'screen'}, {'feature': 'phones'}, {'feature': 'camera'}, {'feature': 'card'}, {'feature': 'service'}, {'feature': 'price'}, {'feature': 'work'}, {'feature': 'quality'}, {'feature': 'problem'}, {'feature': 'life'}, {'feature': 'iphone'}, {'feature': 'product'}, {'feature': 'day'}, {'feature': 'thing'}, {'feature': 'features'}, {'feature': 'cell'}, {'feature': 'way'}, {'feature': 'minutes'}, {'feature': 'everything'}, {'feature': 'days'}, {'feature': 'samsung'}, {'feature': 'works'}, {'feature': 'call'}, {'feature': 'problems'}, {'feature': 'apps'}, {'feature': 'data'}, {'feature': 'device'}, {'feature': 'people'}, {'feature': 'lot'}, {'feature': 'get'}, {'feature': 'months'}, {'feature': 'money'}, {'feature': 'calls'}, {'feature': 'memory'}, {'feature': 'keyboard'}, {'feature': 'music'}, {'feature': 'plan'}, {'feature': 'times'}, {'feature': 'seller'}, {'feature': 'charge'}, {'feature': '

In [87]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
try:
    mydb.features.drop()
finally:
    result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [88]:
my_collection.insert_many(features)